# FingerTrees
##### (http://andrew.gibiansky.com/blog/haskell/finger-trees/)

Finger trees are based on 2-3 trees which have the following properties
* each node has 2 or 3 children
* all leaves are at the same depth

To get from a 2-3 tree to a FingerTree, pinch the leftmost and rightmost nodes and pull them up and together. The resulting structure has these properties
* we pair up the nodes that get pinched together and call this the *spine*
* at each position along the spine we have *prefix* on the left and a *suffix* on the right as well as a link further down the spine
* at the first position of the spine, the prefix and suffix will contain 2 or 3 values that are trees of depth 0 (leaves), the second position will have trees of depth 1, third position will have depth 2 etc.
* the original root of the 2-3 tree now resides at the bottom of the spine

Define the 2-3 tree structure that will be used to store things hanging off the spine:

In [2]:
data Node a = Branch3 a a a
            | Branch2 a a
            deriving Show

By parameterizing the type constructor this way, we effectively guarantee uniform depth; the depth is represented in the type of the tree:

In [5]:
Branch2 'n' 'o'
Branch3 'y' 'e' 's'

let t = Branch2 (Branch3 'a' 'b' 'c') (Branch2 'd' 'e')
t
:t t

Branch2 'n' 'o'

Branch3 'y' 'e' 's'

Branch2 (Branch3 'a' 'b' 'c') (Branch2 'd' 'e')

t :: Node (Node Char)

Implement some utility functions that let us treat `Node` values as a list of 2 or 3 elements. The `OverloadedLists` extension allows us to implement `fromList` and `toList` for different data types and use pattern matching on them as if they were lists:

In [7]:
:ext OverloadedLists
:ext TypeFamilies

import GHC.Exts (IsList(..))

instance IsList (Node a) where
  type Item (Node a) = a
  
  toList (Branch2 x y) = [x, y]
  toList (Branch3 x y z) = [x, y, z]
  
  fromList [x, y] = Branch2 x y
  fromList [x, y, z] = Branch3 x y z
  fromList _ = error "Must have 2 or 3 elements"

Now we need a way to represent the prefix and suffix at each level of the spine. If we are being completely analogous to the 2-3 tree then only the two nodes at the first level of the spine will have 2 or 3 children and itermediate levels will have 1 or 2, the other being the link up the spine. For the sake of uniformity we will just say that each prefix/suffix has between 1 and 4 items. Instead of using a list, we will use a type that restricts to these amounts:

In [9]:
data Affix a = One a
             | Two a a
             | Three a a a
             | Four a a a a
             deriving Show
             
-- Treat them as lists to make things easier
instance IsList (Affix a) where
  type Item (Affix a) = a
  
  toList (One x) = [x]
  toList (Two x y) = [x, y]
  toList (Three x y z) = [x, y, z]
  toList (Four x y z w) = [x, y, z, w]
  
  fromList [x] = One x
  fromList [x, y] = Two x y
  fromList [x, y, z] = Three x y z
  fromList [x, y, z, w] = Four x y z w
  fromList _ = error "Must have 1-4 items"
  
-- The following could be much more efficient, we use the simplest implementation possible
affixPrepend :: a -> Affix a -> Affix a
affixPrepend x = fromList . (x :) . toList

affixAppend :: a -> Affix a -> Affix a
affixAppend x = fromList . (++ [x]) . toList

Now we're ready to create the spine data structure, the `FingerTree` itself:

In [17]:
data FingerTree a
  = Empty -- we can have empty trees
  | Single a -- special singleton case
  | Deep 
    { prefix :: Affix a             -- values to the left
    , deeper :: FingerTree (Node a) -- descending a level results in tree with a greater depth
    , suffix :: Affix a }           -- values to the right
  deriving Show
  
-- example tree

layer3 :: FingerTree a
layer3 = Empty

layer2 :: FingerTree (Node Char)
layer2 = Deep prefix layer3 suffix where
  prefix = [['i', 's'], Branch2 'i' 's']
  suffix = [Branch3 'n' 'o' 't', Branch2 'a' 't']
  
layer1 :: FingerTree Char
layer1 = Deep prefix layer2 suffix where
  prefix = ['t', 'h']
  suffix = ['r', 'e', 'e']
  
exampleTree = layer1

exampleTree

Deep {prefix = Two 't' 'h', deeper = Deep {prefix = Two (Branch2 'i' 's') (Branch2 'i' 's'), deeper = Empty, suffix = Two (Branch3 'n' 'o' 't') (Branch2 'a' 't')}, suffix = Three 'r' 'e' 'e'}

## Finger Trees as Sequences
### Prepend and Append
To append an element to a finger tree we want to add it to the prefix. This won't work if the prefix already has 4 elements, so what we do is take the 5 elements and turn three of them into a `Branch3` and append that to the deeper finger tree:

In [18]:
infixr 5 <|
(<|) :: a -> FingerTree a -> FingerTree a

x <| Empty = Single x

x <| Single y = Deep [x] Empty [y]

x <| Deep [a, b, c, d] deeper suffix = Deep [x, a] (node <| deeper) suffix where
  node = Branch3 b c d
  
x <| tree = tree { prefix = affixPrepend x $ prefix tree }

Prepending follows suit:

In [29]:
infixl 4 |>
(|>) :: FingerTree a -> a -> FingerTree a
Empty |> x = Single x
Single x |> y = Deep [x] Empty [y]
Deep prefix deeper [a, b, c, d] |> x = Deep prefix (deeper |> node) [d, x] where
  node = Branch3 a b c
tree |> y = tree { suffix = affixAppend y $ suffix tree }

In [30]:
't' <| Empty |> 'x' |> 'y' |> 'z' |> 'w' |> 'm'

Deep {prefix = One 't', deeper = Single (Branch3 'x' 'y' 'z'), suffix = Two 'w' 'm'}

## Views (First and Last)
The naive implementation looks like this:

In [32]:
data View a = Nil | View a (FingerTree a) deriving Show

viewl :: FingerTree a -> View a
viewl Empty = Nil
viewl (Single x) = View x Empty
viewl (Deep prefix deeper suffix) =
  View first $ Deep (fromList rest) deeper suffix 
  where
    first:rest = toList prefix
    
-- here's the problem:
let View _ rest = viewl exampleTree
viewl rest

Must have 1-4 items
CallStack (from HasCallStack):
  error, called at <interactive>:18:16 in interactive:Ghci185

Instead we must explicitly check the number of elements in the prefix and if there's only one, we need to recursively grab a `Node a` from the deeper tree so we can return the single element and replace it.

In [38]:
viewl :: FingerTree a -> View a
viewl Empty = Nil
viewl (Single x) = View x Empty
viewl (Deep [x] deeper suffix) = View x rest where
  rest =
    case viewl deeper of
      View node rest' ->
        Deep (fromList $ toList node) rest' suffix
      Nil -> case suffix of
        [x] -> Single x
        [x, y] -> Deep [x] Empty [y]
        [x, y, z] -> Deep [x, y] Empty [z]
        [x, y, z, w] -> Deep [x, y, z] Empty [w]
viewl (Deep prefix deeper suffix) = View x $ Deep (fromList rest) deeper suffix
  where x : rest = toList prefix
  
let View _ rest = viewl exampleTree
viewl rest

View 'h' (Deep {prefix = Two 'i' 's', deeper = Deep {prefix = One (Branch2 'i' 's'), deeper = Empty, suffix = Two (Branch3 'n' 'o' 't') (Branch2 'a' 't')}, suffix = Three 'r' 'e' 'e'})

Right view is implemented the same way:

In [39]:
viewr :: FingerTree a -> View a
viewr Empty = Nil
viewr (Single x) = View x Empty
viewr (Deep prefix deeper [x]) = View x rest where
  rest =
    case viewr deeper of
      View node rest' ->
        Deep prefix rest' (fromList $ toList node)
      Nil -> case prefix of
        [x] -> Single x
        [x, y] -> Deep [x] Empty [y]
        [x, y, z] -> Deep [x] Empty [y, z]
        [x, y, z, w] -> Deep [x] Empty [y, z, w]
viewr (Deep prefix deeper suffix) = View x $ Deep prefix deeper (fromList rest)
  where x : rest = toList suffix

We can now easily convert between FingerTree and List

In [44]:
instance IsList (FingerTree a) where
  type Item (FingerTree a) = a
  
  toList tree = case viewl tree of
    Nil -> []
    View x xs -> x : toList xs
    
  fromList = foldr (<|) Empty
  
[1..6] :: FingerTree Int

Deep {prefix = Two 1 2, deeper = Single (Branch3 3 4 5), suffix = One 6}

## Concatenation
Here's the naive implementation:

In [45]:
(><) :: FingerTree a -> FingerTree a -> FingerTree a
left >< Empty = left
left >< right =
  let View first rest = viewl right in
    (left |> first) >< rest

We can do better than this O(n) runtime by utilising the structure of the FingerTree. First we need a utility function called `nodes` which converts from a list of items to a list `Node`s of items:

In [46]:
nodes :: [a] -> [Node a]
nodes xs = case xs of
  [] -> error "empty list"
  [x] -> error "single element"
  [x, y] -> [Branch2 x y]
  [x, y, z] -> [Branch3 x y z]
  x : y : r -> Branch2 x y : nodes r

If nodes is given a list of n elements it will emit a list of n/2 nodes.

Next we'll redefine concatenation using a somewhat strange operator called `concatWithMiddle` which takes two FingerTrees and as well as a list of elements to stick between them:

In [70]:
(><) :: FingerTree a -> FingerTree a -> FingerTree a
left >< right = concatWithMiddle left [] right

concatWithMiddle :: FingerTree a -> [a] -> FingerTree a -> FingerTree a
concatWithMiddle Empty [] right = right
concatWithMiddle Empty (x:xs) right = x <| concatWithMiddle Empty xs right
concatWithMiddle (Single y) xs right = y <| concatWithMiddle Empty xs right

concatWithMiddle left [] Empty = left
concatWithMiddle left xs Empty = concatWithMiddle left (init xs) Empty |> last xs
concatWithMiddle left xs (Single y) = concatWithMiddle left xs Empty |> y

concatWithMiddle left mid right = Deep (prefix left) deeper' (suffix right) where
  deeper' = concatWithMiddle (deeper left) mid' (deeper right)
  mid' = nodes $ (toList $ suffix left) ++ mid ++ (toList $ prefix right)
  
exampleTree >< exampleTree

Deep {prefix = Two 't' 'h', deeper = Deep {prefix = Two (Branch2 'i' 's') (Branch2 'i' 's'), deeper = Deep {prefix = Two (Branch2 (Branch3 'n' 'o' 't') (Branch2 'a' 't')) (Branch2 (Branch2 'r' 'e') (Branch3 'e' 't' 'h')), deeper = Empty, suffix = One (Branch2 (Branch2 'i' 's') (Branch2 'i' 's'))}, suffix = Two (Branch3 'n' 'o' 't') (Branch2 'a' 't')}, suffix = Three 'r' 'e' 'e'}